In [1]:
import correctionlib
import pandas as pd
import numpy as np
import correctionlib.schemav2 as cs
import hist
import correctionlib.convert
import rich

import uproot

import requests

In [47]:

def get_cset(name, url):
    base_url = 'https://raw.githubusercontent.com/latinos/LatinoAnalysis/UL_production/NanoGardener/python/data/trigger/'
    
    with open('test_sf.txt', 'wb') as file:
        file.write(requests.get(base_url + url).content)
    columns = 'eff stat_d stat_u syst_d syst_u'
    columns = columns.split(' ')
    df = pd.read_csv('test_sf.txt', sep='\t', skiprows=0, header=None, names=['eta_l', 'eta_h', 'pt_l', 'pt_h'] + columns)
    for var in ['eta', 'pt']:
        df[var] = list(zip(df[var + '_l'], df[var + '_h']))
        del df[var + '_l']
        del df[var + '_h']
    eta_bin = np.array(list(map(lambda k: k[0], np.unique(df['eta']))) + [np.unique(df['eta'])[-1][-1]])
    pt_bin = np.array(list(map(lambda k: k[0], np.unique(df['pt']))) + [np.unique(df['pt'])[-1][-1]])
    h = hist.Hist(
        hist.axis.StrCategory(columns, name='syst'),
        hist.axis.Variable(eta_bin, name='eta'),
        hist.axis.Variable(pt_bin, name='pt'),
        hist.storage.Double()
    )
    for syst in columns:
        for pt_c in (pt_bin[1:] + pt_bin[:-1])/2:
            for eta_c in (eta_bin[1:] + eta_bin[:-1])/2:
                for i, (ipt, ieta) in enumerate(list(zip(list(df['pt']), list(df['eta'])))):
                    if (
                        ieta[0] <= eta_c < ieta[1]
                        and ipt[0] <= pt_c < ipt[1]
                    ):
                        h[hist.loc(syst), hist.loc(eta_c), hist.loc(pt_c)] = df[syst][i]
    h.name = name
    h.label='out'
    cset = correctionlib.convert.from_histogram(h)
    return cset

In [80]:
def get_cset_drll(name, trigger_names, urls):
    base_url = 'https://raw.githubusercontent.com/latinos/LatinoAnalysis/UL_production/NanoGardener/python/data/trigger/'

    for trigger_name, url in zip(trigger_names, urls):
        with open(f'test_sf_{trigger_name}.txt', 'wb') as file:
            file.write(requests.get(base_url + url).content)
        with open(f'test_sf_{trigger_name}.txt') as file:
            lines = file.read().split('\n')
        lines = list(map(lambda k: k.strip(), lines))
        with open(f'test_sf_{trigger_name}.txt', 'w') as file:
            file.write('\n'.join(lines))
    
    big_df = {}
    for trigger_name in trigger_names:
        columns = 'sf'
        columns = columns.split(' ')
        df = pd.read_csv(f'test_sf_{trigger_name}.txt', sep=' ', index_col=False, skiprows=0, header=None, names=['drll_l', 'drll_h'] + columns)
        for var in ['drll']:
            df[var] = list(zip(df[var + '_l'], df[var + '_h']))
            del df[var + '_l']
            del df[var + '_h']
        drll_bin = np.array(list(map(lambda k: k[0], np.unique(df['drll']))) + [np.unique(df['drll'])[-1][-1]])
        big_df[trigger_name] = df
    
    h = hist.Hist(
        hist.axis.StrCategory(trigger_names, name='Trigger'),
        hist.axis.Variable(drll_bin, name='drll'),
        hist.storage.Double()
    )
    
    for trigger in trigger_names:
        for drll_c in (drll_bin[1:] + drll_bin[:-1])/2:
            for i, (idrll) in enumerate(list(df['drll'])):
                if (
                    idrll[0] <= drll_c < idrll[1]
                ):
                    h[hist.loc(trigger), hist.loc(drll_c)] = big_df[trigger]['sf'][i]
    h.name = name
    h.label='out'
    cset = correctionlib.convert.from_histogram(h)
    return cset

In [41]:
import sys

In [42]:
sys.path.insert(0, '/gwpool/users/gpizzati/test_processor/my_processor')

In [43]:
from data.common.TrigMaker_cfg import Trigger

DoubleEle
DoubleMu
MuEle
EleMu


📈 DRll_SF (v0)
No description
Node counts: Category: 1, Binning: 4
╭───────────────── ▶ input ─────────────────╮ ╭───── ▶ input ──────╮
│ Trigger (string)                          │ │ drll (real)        │
│ Trigger                                   │ │ drll               │
│ Values: DoubleEle, DoubleMu, EleMu, MuEle │ │ Range: [0.0, 20.0) │
╰───────────────────────────────────────────╯ ╰────────────────────╯
╭─── ◀ output ───╮
│ out (real)     │
│ No description │
╰────────────────╯

In [ ]:
for name, url in Trigger['Full2018v9'][1]['DRllSF'].items():
    csets.append(get_cset_drll('DRll_SF' + name, url))   

In [51]:
csets = []
for name, url in Trigger['Full2018v9'][1]['LegEffData'].items():
    csets.append(get_cset('LegEffData_' + name, url))   
for name, url in Trigger['Full2018v9'][1]['LegEffMC'].items():
    csets.append(get_cset('LegEffMC_' + name, url))   
    
csets.append(get_cset_drll('DRll_SF', list(Trigger['Full2018v9'][1]['DRllSF'].keys()),list(Trigger['Full2018v9'][1]['DRllSF'].values())))

In [52]:
cset = correctionlib.schemav2.CorrectionSet(
    schema_version=2,
    description="",
    corrections=csets
)

In [53]:
rich.print(cset)

CorrectionSet (schema v2)
No description
📂
├── 📈 Data_DoubleEleLegHigPt (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.5, 2.5) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
├── 📈 Data_DoubleEleLegLowPt (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.5, 2.5) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
├── 📈 Data_SingleEle (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.5, 2.5) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
├── 📈 Data_DoubleMuLegHigPt (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.4, 2.4) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
├── 📈 Data_DoubleMuLegLowPt (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.4, 2.4) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
├── 📈 Data_SingleMu (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 5
│   ╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│   │ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│   │ syst                                        │ │ eta                │ │ pt                  │
│   │ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.4, 2.4) │ │ Range: [0.0, 200.0) │
│   ╰─────────────────────────────────────────────╯

In [ ]:
csets[1])

In [37]:
rich.print(cset)

📈 DoubleEleLegHigPt (v0)
No description
Node counts: Category: 1, MultiBinning: 5
╭────────────────── ▶ input ──────────────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ──────╮
│ syst (string)                               │ │ eta (real)         │ │ pt (real)           │
│ syst                                        │ │ eta                │ │ pt                  │
│ Values: eff, stat_d, stat_u, syst_d, syst_u │ │ Range: [-2.5, 2.5) │ │ Range: [0.0, 200.0) │
╰─────────────────────────────────────────────╯ ╰────────────────────╯ ╰─────────────────────╯
╭─── ◀ output ───╮
│ out (real)     │
│ No description │
╰────────────────╯

In [26]:
ceval = cset.to_evaluator()
vpt = 100.0
veta = -2.3
syst = 'eff'
print(ceval.evaluate(syst, veta, vpt))
for i, (ipt, ieta) in enumerate(list(zip(list(df['pt']), list(df['eta'])))):
    if (
        ieta[0] <= veta < ieta[1]
        and ipt[0] <= vpt < ipt[1]
    ):
        print(i)
        print(df[syst][i])

0.985
12
0.985


In [ ]:
cset = correctionlib.convert.from_txt('test_sf.root:h_Ele_signal_pt_eta_bin')

In [3]:
url_base = 'https://github.com/latinos/LatinoAnalysis/raw/UL_production/NanoGardener/python/data/'
url = 'fake_prompt_rates/Full2018v9/mvaFall17V2Iso_WP90/ElePR.root'
with open('test_sf.root', 'wb') as file:
    file.write(requests.get(url_base + url).content)

cset = correctionlib.convert.from_uproot_THx('test_sf.root:h_Ele_signal_pt_eta_bin')

ceval = cset.to_evaluator()

ceval.evaluate(17.0, 0.0)

0.8799667219594568

In [ ]:
dir(uproot.source.xrootd.XRootDSource)

In [10]:
fname = 'root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/mc/RunIISummer20UL18NanoAODv9/EWK_LLJJ_MLL-50_MJJ-120_TuneCP5_13TeV-madgraph-pythia8_dipole/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/40000/3BF5756A-131F-8C4C-9CFC-71B15EDB9351.root'
tree = uproot.open(fname, handler=uproot.source.xrootd.XRootDSource, num_workers=2)['Events']
def branch(name):
    return tree[name].array(entry_stop=10_000)

branch('EMTFbug_veto')

In [16]:
import uproot
import awkward as ak
from copy import deepcopy
import sys
import time
import gc


def read_array(tree, branch_name, start, stop):
    interp = tree[branch_name].interpretation
    interp._forth = True
    return tree[branch_name].array(
        interp,
        entry_start=start,
        entry_stop=stop,
        decompression_executor=uproot.source.futures.TrivialExecutor(),
        interpretation_executor=uproot.source.futures.TrivialExecutor(),
    )
def read_events(filename, start=0, stop=100, read_form={}):
    print("start reading")
    f = uproot.open(filename, handler=uproot.source.xrootd.XRootDSource, num_workers=2)
    tree = f["Events"]
    start = min(start, tree.num_entries)
    stop = min(stop, tree.num_entries)
    if start >= stop:
        return ak.Array([])
    events = {}
    form = deepcopy(read_form)
    for coll in form:
        # print(coll)
        d = {}
        coll_branches = form[coll].pop("branches")
        # print(coll_branches)

        if len(coll_branches) == 0:
            # print(f"Collection {coll} is not a collection but a signle branch")
            events[coll] = read_array(tree, coll, start, stop)
            continue

        for branch in coll_branches:
            d[branch] = read_array(tree, coll + "_" + branch, start, stop)

        if len(d.keys()) == 0:
            print("did not find anything for", coll, file=sys.stderr)
            continue

        events[coll] = ak.zip(d, **form[coll])
        del d

    # f.close()
    print("created events")
    _events = ak.zip(events, depth_limit=1)
    del events
    return _events

In [17]:
fname = 'root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/mc/RunIISummer20UL18NanoAODv9/EWK_LLJJ_MLL-50_MJJ-120_TuneCP5_13TeV-madgraph-pythia8_dipole/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/40000/3BF5756A-131F-8C4C-9CFC-71B15EDB9351.root'
events = read_events(fname, stop=10_000, read_form=forms['mc'])

start reading
created events


In [25]:
cuts_muon = {
                             'cuts' : { 
                                # Common cuts
                                'True' :
                                 [
                                   'abs(muon_col["eta"]) < 2.4' , 
                                   'muon_col["tightId"] == 1' ,
                                   'abs(muon_col["dz"]) < 0.1' ,
                                   'muon_col["pfRelIso04_all"] < 0.4',
                                 ] ,
                                 # dxy for pT < 20 GeV
                                 'muon_col["pt"] <= 20.0' :
                                 [
                                    'abs(muon_col["dxy"]) < 0.01 ' ,
                                 ] ,
                                 # dxy for pT > 20 GeV
                                 'muon_col["pt"] > 20.0' :
                                 [
                                    'abs(muon_col["dxy"]) < 0.02 ' ,
                                 ] ,
                                  } ,}

In [56]:
MuonWP ={ 'VetoObjWP' : { 
      'HLTsafe' : {
                         'cuts' : { 
                                # Common cuts
                                'True' :
                                 [
                                   'abs(muon_col[LF_idx]["eta"]) < 2.4' , 
                                   'muon_col[LF_idx]["pt"] > 10.0' ,
                                 ]
                                  } ,
                   }
               } ,

 # ------------ 
 'FakeObjWP'  : {

      'HLTsafe' : {
                         'cuts' : { 
                                # Common cuts
                                'True' :
                                 [
                                   'abs(muon_col[LF_idx]["eta"]) < 2.4' , 
                                   'muon_col[LF_idx]["tightId"] == 1' ,
                                   'abs(muon_col[LF_idx]["dz"]) < 0.1' ,
                                   'muon_col[LF_idx]["pfRelIso04_all"] < 0.4',
                                 ] ,
                                 # dxy for pT < 20 GeV
                                 'muon_col[LF_idx]["pt"] <= 20.0' :
                                 [
                                    'abs(muon_col[LF_idx]["dxy"]) < 0.01 ' ,
                                 ] ,
                                 # dxy for pT > 20 GeV
                                 'muon_col[LF_idx]["pt"] > 20.0' :
                                 [
                                    'abs(muon_col[LF_idx]["dxy"]) < 0.02 ' ,
                                 ] ,
                                  } ,

                       } ,
                 
                 } ,

 # ------------ 
 'TightObjWP' :  {

      'cut_Tight_HWWW' : {
                         'cuts' : { 
                                # Common cuts
                                'True' :
                                 [ 
                                   'abs(muon_col[LF_idx]["eta"]) < 2.4' ,
                                   'muon_col[LF_idx]["tightId"] == 1' ,
                                   'abs(muon_col[LF_idx]["dz"]) < 0.1' ,
                                   'muon_col[LF_idx]["pfRelIso04_all"] < 0.15',
                                 ] ,
                                 # dxy for pT < 20 GeV
                                 'muon_col[LF_idx]["pt"] <= 20.0' :
                                 [
                                    'abs(muon_col[LF_idx]["dxy"]) < 0.01 ' ,
                                 ] ,
                                 # dxy for pT > 20 GeV
                                 'muon_col[LF_idx]["pt"] > 20.0' :
                                 [
                                    'abs(muon_col[LF_idx]["dxy"]) < 0.02 ' ,
                                 ] ,
                                  } ,
                         'idSF':  {
                                    '1-1' : [ 'LatinoAnalysis/NanoGardener/python/data/scale_factor/Full2018v9/NUM_TightHWW_DEN_TrackerMuons_eta_pt.root'],
                                  } ,
                         'isoSF': {
                                    '1-1' : [ 'LatinoAnalysis/NanoGardener/python/data/scale_factor/Full2018v9/NUM_TightHWW_ISO_DEN_TightHWW_eta_pt.root'],
                                  } ,
                         'fakeW' : '/LatinoAnalysis/NanoGardener/python/data/fake_prompt_rates/Full2018v9/cut_Tight_HWWW/',

                       } ,
 }}



In [57]:
MuonWP['TightObjWP']['cut_Tight_HWWW']

{'cuts': {'True': ['abs(muon_col[LF_idx]["eta"]) < 2.4',
   'muon_col[LF_idx]["tightId"] == 1',
   'abs(muon_col[LF_idx]["dz"]) < 0.1',
   'muon_col[LF_idx]["pfRelIso04_all"] < 0.15'],
  'muon_col[LF_idx]["pt"] <= 20.0': ['abs(muon_col[LF_idx]["dxy"]) < 0.01 '],
  'muon_col[LF_idx]["pt"] > 20.0': ['abs(muon_col[LF_idx]["dxy"]) < 0.02 ']},
 'idSF': {'1-1': ['LatinoAnalysis/NanoGardener/python/data/scale_factor/Full2018v9/NUM_TightHWW_DEN_TrackerMuons_eta_pt.root']},
 'isoSF': {'1-1': ['LatinoAnalysis/NanoGardener/python/data/scale_factor/Full2018v9/NUM_TightHWW_ISO_DEN_TightHWW_eta_pt.root']},
 'fakeW': '/LatinoAnalysis/NanoGardener/python/data/fake_prompt_rates/Full2018v9/cut_Tight_HWWW/'}

In [26]:
cuts_muon['cuts']

{'True': ['abs(muon_col["eta"]) < 2.4',
  'muon_col["tightId"] == 1',
  'abs(muon_col["dz"]) < 0.1',
  'muon_col["pfRelIso04_all"] < 0.4'],
 'muon_col["pt"] <= 20.0': ['abs(muon_col["dxy"]) < 0.01 '],
 'muon_col["pt"] > 20.0': ['abs(muon_col["dxy"]) < 0.02 ']}

In [65]:
import sys

In [67]:
sys.path.insert(0, '/gwpool/users/gpizzati/test_processor/my_processor')
sys.path.insert(0, '/gwpool/users/gpizzati/test_processor/my_processor/data/2018')

In [68]:
import LeptonSel

In [70]:
year = 'Full2018v9'
ElectronWP = LeptonSel.ElectronWP[year]
MuonWP = LeptonSel.MuonWP[year]

In [72]:
ElectronWP

{'VetoObjWP': {'HLTsafe': {'cuts': {'True': ['False']}}},
 'FakeObjWP': {'HLTsafe': {'cuts': {'True': ['abs(electron_col[LF_idx]["eta"]) < 2.5',
     'electron_col[LF_idx]["cutBased"] >= 3',
     'electron_col[LF_idx]["convVeto"] == 1'],
    'abs(electron_col[LF_idx]["eta"]) <= 1.479': ['abs(electron_col[LF_idx]["dxy"]) < 0.05',
     'abs(electron_col[LF_idx]["dz"]) < 0.1'],
    'abs(electron_col[LF_idx]["eta"]) > 1.479': ['electron_col[LF_idx]["sieie"] < 0.03 ',
     'abs(electron_col[LF_idx]["eInvMinusPInv"]) < 0.014',
     'abs(electron_col[LF_idx]["dxy"]) < 0.1',
     'abs(electron_col[LF_idx]["dz"]) < 0.2']}}},
 'TightObjWP': {'mvaFall17V2Iso_WP90': {'cuts': {'True': ['abs(electron_col[LF_idx]["eta"]) < 2.5',
     'electron_col[LF_idx]["mvaFall17V2Iso_WP90"]',
     'electron_col[LF_idx]["convVeto"] == 1',
     'electron_col[LF_idx]["pfRelIso03_all"] < 0.06'],
    'abs(electron_col[LF_idx]["eta"]) <= 1.479': ['abs(electron_col[LF_idx]["dxy"]) < 0.05',
     'abs(electron_col[LF_idx]

In [76]:
ak.none_like

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


AttributeError: module 'awkward' has no attribute 'none_like'

In [75]:
ak.concatenate([ak.local_index(events.Electron)], axis=1)

<Array [[], [], [0], [0], ..., [], [0, 1, 2], []] type='10000 * var * int64'>

In [62]:
def test(events):
    electron_col = events.Electron
    
    muon_col = events.Muon
    for wp in MuonWP['FakeObjWP']:
        comb = ak.ones_like(muon_col.pt) == 1.0
        for key, cuts in MuonWP['FakeObjWP'][wp]['cuts'].items():
            tmp1 = eval(key.replace("[LF_idx]", ""))
            tmp2 = ak.ones_like(muon_col.pt) == 1.0
            for cut in cuts:
                tmp2 = tmp2 & eval(cut.replace("[LF_idx]", ""))
            comb = comb & (~tmp1 | tmp2)
        muon_col['isLoose'] = ak.values_astype(comb, bool)
        
    for wp in MuonWP['TightObjWP']:
        comb = ak.ones_like(muon_col.pt) == 1.0
        for key, cuts in MuonWP['TightObjWP'][wp]['cuts'].items():
            tmp1 = eval(key.replace("[LF_idx]", ""))
            tmp2 = ak.ones_like(muon_col.pt) == 1.0
            for cut in cuts:
                tmp2 = tmp2 & eval(cut.replace("[LF_idx]", ""))
            comb = comb & (~tmp1 | tmp2)
        muon_col['isTightMuon_' + wp] = ak.values_astype(comb, bool)
        
    return muon_col

In [63]:
events.Muon[test(events).isTightMuon_cut_Tight_HWWW].pt

<Array [[27.5], [], [], [], ..., [], [], [], []] type='10000 * var * float32'>

In [64]:
events.Muon[test(events).isLoose].pt

<Array [[27.5], [], [], [], ..., [], [], [], []] type='10000 * var * float32'>

KeyInFileError: not found: 'EMTFbug_veto'

    Available keys: 'Electron_eta', 'Electron_dz', 'Electron_pt', 'Electron_r9', 'Electron_convVeto', 'Electron_dxy', 'Electron_hoe', 'Electron_phi', 'Electron_eCorr', 'Electron_ip3d', 'Electron_mass', 'Electron_jetIdx', 'Electron_dzErr'...

in file root://ccxrdcms.in2p3.fr:1094/pnfs/in2p3.fr/data/cms/disk/data//store/mc/RunIISummer20UL18NanoAODv9/EWK_LLJJ_MLL-50_MJJ-120_TuneCP5_13TeV-madgraph-pythia8_dipole/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/40000/3BF5756A-131F-8C4C-9CFC-71B15EDB9351.root
in object /Events;1

In [5]:
events = uproot.concatenate({'': 'Events'})


KeyboardInterrupt



In [ ]:
EMTFbug_veto

In [2]:
url = 'https://github.com/latinos/LatinoAnalysis/raw/UL_production/NanoGardener/python/data/scale_factor/Full2018v9/NUM_TightHWW_DEN_TrackerMuons_eta_pt.root'
with open('test_sf.root', 'wb') as file:
    file.write(requests.get(url).content)

In [3]:
f = uproot.open('test_sf.root')

In [4]:
h = f['NUM_TightHWW_DEN_TrackerMuons_eta_pt'].to_hist()

In [5]:
h.axes[0].edges

array([-2.4, -2.1, -1.6, -1.2, -0.8, -0.3, -0.2,  0. ,  0.2,  0.3,  0.8,
        1.2,  1.6,  2.1,  2.4])

In [6]:
axis = [hist.axis.Variable(axis.edges, name=name) for axis, name in zip(h.axes, ['eta', 'pt'])]

In [7]:
axis

[Variable([-2.4, -2.1, -1.6, -1.2, -0.8, -0.3, -0.2, 0, 0.2, 0.3, 0.8, 1.2, 1.6, 2.1, 2.4], name='eta'),
 Variable([10, 15, 20, 25, 30, 40, 60, 100, 200], name='pt')]

In [8]:
data = np.array([h.values(), np.sqrt(h.variances())])

In [9]:
h_sf = hist.Hist(hist.axis.StrCategory(['nominal', 'syst'], name='syst'), *axis, data=data)

In [10]:
h_sf.name = 'Muon SF'
h_sf.label='out'

In [11]:
cset = correctionlib.convert.from_histogram(h_sf)

In [12]:
rich.print(cset)

📈 Muon SF (v0)
No description
Node counts: Category: 1, MultiBinning: 2
╭─────── ▶ input ───────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ───────╮
│ syst (string)         │ │ eta (real)         │ │ pt (real)            │
│ syst                  │ │ eta                │ │ pt                   │
│ Values: nominal, syst │ │ Range: [-2.4, 2.4) │ │ Range: [10.0, 200.0) │
╰───────────────────────╯ ╰────────────────────╯ ╰──────────────────────╯
╭─── ◀ output ───╮
│ out (real)     │
│ No description │
╰────────────────╯

In [13]:
ceval = cset.to_evaluator()
veta = 1.5
vpt = 15.0
point = h[hist.loc(veta), hist.loc(vpt)]
print(point.value, np.sqrt(point.variance))
args = (veta, vpt)
print(ceval.evaluate('nominal', *args), ceval.evaluate('syst', *args))

0.9943716526031494 0.0020353191106662998
0.9943716526031494 0.0020353191106663


In [15]:
cset_muon = cset

In [19]:
with open('test_sf.txt', 'w') as file:
    file.write(requests.get('https://raw.githubusercontent.com/latinos/LatinoAnalysis/UL_production/NanoGardener/python/data/scale_factor/Full2018v9/egammaEffi_TightHWW_2018.txt').text)

In [20]:
columns = 'effData statErrData systErrData effMC statErrMC systErrMC effDataAltBkg effDataAltSig effMCAltMC effMCTagSel'
columns = columns.split(' ')

In [21]:
df = pd.read_csv('test_sf.txt', sep='\t', skiprows=3, header=None, names=['eta_l', 'eta_h', 'pt_l', 'pt_h'] + columns)
df

,eta_l,eta_h,pt_l,pt_h,effData,statErrData,systErrData,effMC,statErrMC,systErrMC,effDataAltBkg,effDataAltSig,effMCAltMC,effMCTagSel
0,-2.500,-2.000,10.0,15.0,0.223,0.004,0.006,0.193,0.005,0.008,0.218,0.221,0.201,0.193
1,-2.000,-1.566,10.0,15.0,0.341,0.005,0.013,0.300,0.006,0.003,0.328,0.344,0.302,0.297
2,-1.566,-1.442,10.0,15.0,0.373,0.015,0.032,0.233,0.012,0.047,0.341,0.378,0.280,0.231
3,-1.442,-0.800,10.0,15.0,0.452,0.006,0.026,0.357,0.006,0.011,0.428,0.442,0.368,0.357
4,-0.800,0.000,10.0,15.0,0.353,0.004,0.014,0.364,0.005,0.006,0.339,0.357,0.358,0.361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.000,0.800,150.0,500.0,0.897,0.004,0.007,0.899,0.006,0.016,0.889,0.896,0.883,0.899
66,0.800,1.442,150.0,500.0,0.887,0.005,0.016,0.882,0.007,0.012,0.872,0.888,0.870,0.885
67,1.442,1.566,150.0,500.0,0.773,0.019,0.027,0.709,0.033,0.033,0.746,0.773,0.739,0.724
68,1.566,2.000,150.0,500.0,0.883,0.007,0.011,0.893,0.011,0.009,0.872,0.883,0.886,0.899


In [22]:
for var in ['eta', 'pt']:
    df[var] = list(zip(df[var + '_l'], df[var + '_h']))
    del df[var + '_l']
    del df[var + '_h']

In [23]:
df['sf'] = df['effData']/df['effMC']
df['sf_err'] = np.sqrt( (df['statErrData']/df['effData'])**2 + (df['statErrMC']/df['effMC'])**2 ) * df['sf']
df['sf_syst'] = np.sqrt( (df['systErrData']/df['effData'])**2 + (df['systErrMC']/df['effMC'])**2 ) * df['sf']

In [24]:
eta_bin = np.array(list(map(lambda k: k[0], np.unique(df['eta']))) + [np.unique(df['eta'])[-1][-1]])
pt_bin = np.array(list(map(lambda k: k[0], np.unique(df['pt']))) + [np.unique(df['pt'])[-1][-1]])
print(eta_bin)
print(pt_bin)

[-2.5   -2.    -1.566 -1.442 -0.8    0.     0.8    1.442  1.566  2.
  2.5  ]
[ 10.  15.  20.  35.  50.  90. 150. 500.]


In [26]:
h = hist.Hist(
    hist.axis.StrCategory(['nominal', 'stat', 'syst'], name='syst'),
    hist.axis.Variable(eta_bin, name='eta'),
    hist.axis.Variable(pt_bin, name='pt'),
    hist.storage.Double()
)

In [27]:
for syst, sf_name in zip(['nominal', 'stat', 'syst'], ['sf', 'sf_err', 'sf_syst']):
    for pt_c in (pt_bin[1:] + pt_bin[:-1])/2:
        for eta_c in (eta_bin[1:] + eta_bin[:-1])/2:
            for i, (ipt, ieta) in enumerate(list(zip(list(df['pt']), list(df['eta'])))):
                if (
                    ieta[0] <= eta_c < ieta[1]
                    and ipt[0] <= pt_c < ipt[1]
                ):
                    #print(i)
                    #print(df['sf'][i], df['sf_err'][i])
                    h[hist.loc(syst), hist.loc(eta_c), hist.loc(pt_c)] = df[sf_name][i]

In [28]:
h.name = 'Electron SF'
h.label='out'

In [29]:
cset = correctionlib.convert.from_histogram(h)

In [30]:
rich.print(cset)

📈 Electron SF (v0)
No description
Node counts: Category: 1, MultiBinning: 3
╭────────── ▶ input ──────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ───────╮
│ syst (string)               │ │ eta (real)         │ │ pt (real)            │
│ syst                        │ │ eta                │ │ pt                   │
│ Values: nominal, stat, syst │ │ Range: [-2.5, 2.5) │ │ Range: [10.0, 500.0) │
╰─────────────────────────────╯ ╰────────────────────╯ ╰──────────────────────╯
╭─── ◀ output ───╮
│ out (real)     │
│ No description │
╰────────────────╯

In [32]:
ceval = cset.to_evaluator()
vpt = 23.0
veta = +0.1
print(ceval.evaluate('nominal', veta, vpt), ceval.evaluate('stat', veta, vpt))
for i, (ipt, ieta) in enumerate(list(zip(list(df['pt']), list(df['eta'])))):
    if (
        ieta[0] <= veta < ieta[1]
        and ipt[0] <= vpt < ipt[1]
    ):
        print(i)
        print(df['sf'][i], df['sf_err'][i])

0.9565217391304348 0.0020055236878184692
25
0.9565217391304349 0.0020055236878184692


In [33]:
cset_electron = cset

In [34]:
cset = correctionlib.schemav2.CorrectionSet(
    schema_version=2,
    description="",
    corrections=[
        cset_muon,
        cset_electron
    ],
)

In [35]:
rich.print(cset)

CorrectionSet (schema v2)
No description
📂
├── 📈 Muon SF (v0)
│   No description
│   Node counts: Category: 1, MultiBinning: 2
│   ╭─────── ▶ input ───────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ───────╮
│   │ syst (string)         │ │ eta (real)         │ │ pt (real)            │
│   │ syst                  │ │ eta                │ │ pt                   │
│   │ Values: nominal, syst │ │ Range: [-2.4, 2.4) │ │ Range: [10.0, 200.0) │
│   ╰───────────────────────╯ ╰────────────────────╯ ╰──────────────────────╯
│   ╭─── ◀ output ───╮
│   │ out (real)     │
│   │ No description │
│   ╰────────────────╯
└── 📈 Electron SF (v0)
    No description
    Node counts: Category: 1, MultiBinning: 3
    ╭────────── ▶ input ──────────╮ ╭───── ▶ input ──────╮ ╭────── ▶ input ───────╮
    │ syst (string)               │ │ eta (real)         │ │ pt (real)            │
    │ syst                        │ │ eta                │ │ pt                   │
    │ Values: nominal, stat, syst │ │ Range: [-2.5, 2.5) │ │ Range: [10.0, 500.0) │
    ╰─────────────────────────────╯ ╰────────────────────╯ ╰──────────────────────╯
    ╭─── ◀ output ───╮
    │ out (real)     │
    │ No description │
    ╰────────────────╯

In [39]:
!pwd

/gwpool/users/gpizzati


In [38]:
!ls testcoffea/AJJ_coffea_DNN/ajj/data/2018

btagging.json
btagging.json.gz
egamma.ea.txt
EGM_ScaleUnc.json.gz
electron.json.gz
jmar.json.gz
muon_Z.json.gz
new_unc.junc.txt
new_unc.txt
puWeights.json.gz
Regrouped_Summer19UL17_V5_MC_UncertaintySources_AK4PFchs.junc.txt
Summer19UL18_JRV2_MC_PtResolution_AK4PFchs.jr.txt
Summer19UL18_JRV2_MC_SF_AK4PFchs.jersf.txt
Summer19UL18_V5_MC_L1FastJet_AK4PFchs.jec.txt
Summer19UL18_V5_MC_L2L3Residual_AK4PFchs.jec.txt
Summer19UL18_V5_MC_L2Relative_AK4PFchs.jec.txt
Summer19UL18_V5_MC_L3Absolute_AK4PFchs.jec.txt
Summer19UL18_V5_MC_Uncertainty_AK4PFchs.junc.txt


In [40]:
import gzip

with gzip.open("/gwpool/users/gpizzati/testcoffea/AJJ_coffea_DNN/ajj/data/2018/lepton_sf.json.gz", "wt") as fout:
    fout.write(cset.json(exclude_unset=True))

In [43]:
type(cset)

correctionlib.schemav2.CorrectionSet

In [46]:
correctionlib.schemav2.from_file

AttributeError: module 'correctionlib.schemav2' has no attribute 'from_file'

In [15]:
import json
string = """
{
    "data": {
        "Jet": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "mass",
                "jetId",
                "puId",
                "rawFactor",
                "area",
                "btagDeepFlavB"
            ],
            "with_name": "Momentum4D"
        },
        "Electron": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "pdgId",
                "mvaFall17V2Iso_WP90",
                "convVeto",
                "pfRelIso03_all",
                "dxy",
                "dz"
            ],
            "with_name": "Momentum4D"
        },
        "Muon": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "pdgId",
                "tightId",
                "pfRelIso04_all",
                "dxy",
                "dz"
            ],
            "with_name": "Momentum4D"
        },
        "fixedGridRhoFastjetAll": {
            "branches": [],
            "with_name": null
        },
        "HLT": {
            "branches": [
                "Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL",
                "Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ",
                "Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8",
                "IsoMu24",
                "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL",
                "Ele32_WPTight_Gsf"
            ],
            "with_name": null
        },
        "Flag": {
            "branches": [
                "goodVertices",
                "globalSuperTightHalo2016Filter",
                "HBHENoiseFilter",
                "HBHENoiseIsoFilter",
                "EcalDeadCellTriggerPrimitiveFilter",
                "BadPFMuonFilter",
                "BadPFMuonDzFilter",
                "ecalBadCalibFilter",
                "eeBadScFilter"
            ],
            "with_name": null
        },
        "run": {
            "branches": [],
            "with_name": null
        },
        "luminosityBlock": {
            "branches": [],
            "with_name": null
        }
    },
    "mc": {
        "Jet": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "mass",
                "jetId",
                "puId",
                "genJetIdx",
                "rawFactor",
                "area",
                "btagDeepFlavB",
                "hadronFlavour"
            ],
            "with_name": "Momentum4D"
        },
        "GenPart": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "mass",
                "pdgId",
                "status",
                "genPartIdxMother",
                "statusFlags"
            ],
            "with_name": "Momentum4D"
        },
        "GenJet": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "mass"
            ],
            "with_name": "Momentum4D"
        },
        "Electron": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "pdgId",
                "mvaFall17V2Iso_WP90",
                "convVeto",
                "pfRelIso03_all",
                "dxy",
                "dz"
            ],
            "with_name": "Momentum4D"
        },
        "Muon": {
            "branches": [
                "pt",
                "eta",
                "phi",
                "pdgId",
                "tightId",
                "pfRelIso04_all",
                "dxy",
                "dz"
            ],
            "with_name": "Momentum4D"
        },
        "fixedGridRhoFastjetAll": {
            "branches": [],
            "with_name": null
        },
        "genWeight": {
            "branches": [],
            "with_name": null
        },
        "LHEScaleWeight": {
            "branches": [],
            "with_name": null
        },
        "LHEPdfWeight": {
            "branches": [],
            "with_name": null
        },
        "HLT": {
            "branches": [
                "Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL",
                "Mu12_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ",
                "Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8",
                "IsoMu24",
                "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL",
                "Ele32_WPTight_Gsf"
            ],
            "with_name": null
        },
        "Flag": {
            "branches": [
                "goodVertices",
                "globalSuperTightHalo2016Filter",
                "HBHENoiseFilter",
                "HBHENoiseIsoFilter",
                "EcalDeadCellTriggerPrimitiveFilter",
                "BadPFMuonFilter",
                "BadPFMuonDzFilter",
                "ecalBadCalibFilter",
                "eeBadScFilter"
            ],
            "with_name": null
        },
        "Pileup": {
            "branches": [
                "nTrueInt"
            ],
            "with_name": null
        }
    }
}
"""
forms = json.loads(string)